In [57]:
import pandas as pd
import numpy as np
import math
import random


#ignore_features = ['58', '16', '7', '25', '58', '23', '20']
ignore_features = []
toencode = {}
f = open("field_types.txt", 'r')
for line in f.readlines():
    line = line.strip().split(" ")
    if line[1] != "numeric" and line[0] not in ignore_features:
        toencode[line[0]] = len(line) - 1
        

f1 = open("data.csv")
data_df = pd.read_csv(f1, sep=',', header = 'infer', low_memory=False)
data_df = data_df.drop(ignore_features, axis=1)
label_df = data_df[['label']]
data_df = data_df.drop('label', axis=1)
label = np.array(label_df)


f2 = open("quiz.csv")
quiz_df = pd.read_csv(f2, sep=',', header = 'infer', low_memory=False)
quiz_df = quiz_df.drop(ignore_features, axis=1)



In [58]:
data_end = np.shape(data_df)[0]
df = data_df.append(quiz_df)
df_trans = pd.get_dummies(df, columns=toencode.keys(), dummy_na=False)
data = np.array(df_trans)

data, quiz = data[:data_end], data[data_end:]

In [59]:
def splitdata(data, label):
    n = data.shape[0]
    train_num = int(math.ceil(n*.8))
    indices = random.sample(xrange(n), n)

    training_idx, test_idx = indices[train_num:], indices[:train_num]
    training, test = data[training_idx,:], data[test_idx,:]
    lab, testlabel = label[training_idx,:], label[test_idx,:]
    
    return training, lab, test, testlabel

train, lab, test, testlabel = splitdata(data, label)

In [60]:
def eval(preds, testlabels):
    if len(preds) == len(testlabels):
        error = np.count_nonzero(preds != testlabels) / np.float(len(preds))
        return 1 - error
    


In [61]:
#preds = knn2(train, lab, test)
#print eval(preds, testlabel)

In [62]:
from sklearn.neighbors import KNeighborsClassifier
lab = np.ravel(lab)
neigh = KNeighborsClassifier(n_neighbors=2, algorithm='ball_tree')
f = neigh.fit(train, lab) 

In [63]:
print np.shape(train)
print np.shape(quiz)
preds = neigh.predict(quiz)

(25367, 5932)
(31709, 5932)


In [64]:
x = []
for i in xrange(len(preds)):
    p = preds[i]
    x.append((i+1, p))

In [65]:
preds_df = pd.DataFrame(x, columns=['Id', 'Prediction'])
preds_df.to_csv('knn_out.csv', index=False)